# Benchmarking

## 0. Setup the logging

This step sets up logging in our environment to increase our visibility over
the steps that GreenGuard performs.

In [1]:
import logging;

logging.basicConfig(level=logging.INFO)
logging.getLogger().setLevel(level=logging.ERROR)
logging.getLogger('greenguard').setLevel(level=logging.INFO)

import warnings
warnings.simplefilter("ignore")


## Running the Benchmarking

The user API for the GreenGuard Benchmarking is the `greenguard.benchmark.evaluate_templates` function.

The `evaluate_templates` function accepts the following arguments:
* `templates (list)`: List of templates to try.
* `window_size_rule (list)`: List of tupples (int, str or Timedelta object).
* `metric (function or str)`: Metric to use. If an ``str`` is give it must be one of the metrics defined in the `greenguard.metrics.METRICS` dictionary.
* `tuning_iterations (int)`: Number of iterations to be used.
* `init_params (dict)`: Initialization parameters for the pipelines.
* `target_times (DataFrame)`: Contains the specefication problem that we are solving, which has three columns:
    * `turbine_id`: Unique identifier of the turbine which this label corresponds to.
    * `cutoff_time`: Time associated with this target.
    * `target`: The value that we want to predict. This can either be a numerical value
        or a categorical label. This column can also be skipped when preparing
        data that will be used only to make predictions and not to fit any
        pipeline.
* `readings (DataFrame)`: Contains the signal data from different sensors, with the following columns:
    * `turbine_id`: Unique identifier of the turbine which this reading comes from.
    * `signal_id`: Unique identifier of the signal which this reading comes from.
    * `timestamp (datetime)`: Time where the reading took place, as a datetime.
    * `value (float)`: Numeric value of this reading.
* `preprocessing (int, list or dict)`: Number of preprocessing steps to be used.
* `cost (bool)`: Wheter the metric is a cost function (the lower the better) or not.
* `test_size (float)`: Percentage of the data set to be used for the test.
* `cv_splits (int)`: Amount of splits to create.
* `random_state (int)`: Random number of train_test split.
* `output_path (str)`: Path where to save the benchmark report.
* `cache_path (str)`: If given, cache the generated cross validation splits in this folder. Defatuls to ``None``.

In [2]:
templates = ['unstack_lstm_timeseries_classifier', 'normalize_dfs_xgb_classifier']
window_size_rule = [('1d', '1h'), ('3d', '4h')]
init_params = {
    'unstack_lstm_timeseries_classifier': {
        'keras.Sequential.LSTMTimeSeriesClassifier#1': {
            'epochs': 1,
        }
    }
}


In [3]:
from greenguard.benchmark import evaluate_templates

results = evaluate_templates(
    templates=templates,
    window_size_rule=window_size_rule,
    init_params=init_params,
    tuning_iterations=3,
    cv_splits=3,
)

Using TensorFlow backend.
INFO:greenguard.pipeline:New configuration found:
  Template: unstack_lstm_timeseries_classifier 
    Hyperparameters: 
      ('sklearn.impute.SimpleImputer#1', 'strategy'): mean
      ('keras.Sequential.LSTMTimeSeriesClassifier#1', 'lstm_1_units'): 80
      ('keras.Sequential.LSTMTimeSeriesClassifier#1', 'dropout_1_rate'): 0.3
      ('keras.Sequential.LSTMTimeSeriesClassifier#1', 'dense_1_units'): 80
INFO:greenguard.pipeline:New configuration found:
  Template: unstack_lstm_timeseries_classifier 
    Hyperparameters: 
      ('sklearn.impute.SimpleImputer#1', 'strategy'): mean
      ('keras.Sequential.LSTMTimeSeriesClassifier#1', 'lstm_1_units'): 80
      ('keras.Sequential.LSTMTimeSeriesClassifier#1', 'dropout_1_rate'): 0.3
      ('keras.Sequential.LSTMTimeSeriesClassifier#1', 'dense_1_units'): 80
INFO:greenguard.pipeline:New configuration found:
  Template: unstack_lstm_timeseries_classifier 
    Hyperparameters: 
      ('sklearn.impute.SimpleImputer#1', 'st

Built 99 features
Elapsed: 02:58 | Progress: 100%|██████████
Elapsed: 00:58 | Progress: 100%|██████████
Built 99 features
Elapsed: 01:54 | Progress: 100%|██████████
Elapsed: 01:08 | Progress: 100%|██████████
Built 99 features
Elapsed: 02:20 | Progress: 100%|██████████
Elapsed: 01:09 | Progress: 100%|██████████
Built 99 features
Elapsed: 02:16 | Progress: 100%|██████████
Elapsed: 01:07 | Progress: 100%|██████████


INFO:greenguard.pipeline:New configuration found:
  Template: normalize_dfs_xgb_classifier 
    Hyperparameters: 
      ('mlprimitives.custom.feature_extraction.CategoricalEncoder#1', 'max_labels'): 0
      ('xgboost.XGBClassifier#1', 'n_estimators'): 100
      ('xgboost.XGBClassifier#1', 'max_depth'): 3
      ('xgboost.XGBClassifier#1', 'learning_rate'): 0.1
      ('xgboost.XGBClassifier#1', 'gamma'): 0.0
      ('xgboost.XGBClassifier#1', 'min_child_weight'): 1
INFO:greenguard.pipeline:New configuration found:
  Template: normalize_dfs_xgb_classifier 
    Hyperparameters: 
      ('mlprimitives.custom.feature_extraction.CategoricalEncoder#1', 'max_labels'): 18
      ('xgboost.XGBClassifier#1', 'n_estimators'): 920
      ('xgboost.XGBClassifier#1', 'max_depth'): 10
      ('xgboost.XGBClassifier#1', 'learning_rate'): 0.02731362750079913
      ('xgboost.XGBClassifier#1', 'gamma'): 0.46258174821600884
      ('xgboost.XGBClassifier#1', 'min_child_weight'): 3


Built 99 features
Elapsed: 03:18 | Progress: 100%|██████████
Elapsed: 01:03 | Progress: 100%|██████████
Built 99 features
Elapsed: 03:15 | Progress: 100%|██████████
Elapsed: 01:06 | Progress: 100%|██████████
Built 99 features
Elapsed: 02:05 | Progress: 100%|██████████
Elapsed: 01:10 | Progress: 100%|██████████
Built 99 features
Elapsed: 01:51 | Progress: 100%|██████████
Elapsed: 00:54 | Progress: 100%|██████████
Built 99 features
Elapsed: 01:51 | Progress: 100%|██████████
Elapsed: 00:58 | Progress: 100%|██████████


INFO:greenguard.pipeline:New configuration found:
  Template: normalize_dfs_xgb_classifier 
    Hyperparameters: 
      ('mlprimitives.custom.feature_extraction.CategoricalEncoder#1', 'max_labels'): 0
      ('xgboost.XGBClassifier#1', 'n_estimators'): 100
      ('xgboost.XGBClassifier#1', 'max_depth'): 3
      ('xgboost.XGBClassifier#1', 'learning_rate'): 0.1
      ('xgboost.XGBClassifier#1', 'gamma'): 0.0
      ('xgboost.XGBClassifier#1', 'min_child_weight'): 1
INFO:greenguard.pipeline:New configuration found:
  Template: normalize_dfs_xgb_classifier 
    Hyperparameters: 
      ('mlprimitives.custom.feature_extraction.CategoricalEncoder#1', 'max_labels'): 7
      ('xgboost.XGBClassifier#1', 'n_estimators'): 348
      ('xgboost.XGBClassifier#1', 'max_depth'): 3
      ('xgboost.XGBClassifier#1', 'learning_rate'): 0.5272082810065426
      ('xgboost.XGBClassifier#1', 'gamma'): 0.04014402178038856
      ('xgboost.XGBClassifier#1', 'min_child_weight'): 2


Built 99 features
Elapsed: 02:42 | Progress: 100%|██████████
Elapsed: 01:00 | Progress: 100%|██████████


In [4]:
results

,template,window_size,resample_rule,default_test,default_cv,tuned_cv,tuned_test,status
0,unstack_lstm_timeseries_classifier,1d,1h,0.711864,0.646437,0.646437,0.666667,OK
1,unstack_lstm_timeseries_classifier,3d,4h,0.703704,0.577295,0.616052,0.709677,OK
2,normalize_dfs_xgb_classifier,1d,1h,0.581818,0.619698,0.646750,0.631579,OK
3,normalize_dfs_xgb_classifier,3d,4h,0.581818,0.619698,0.637957,0.642857,OK
